In [1]:
from teste import (
    TICKERS, NUM_ATIVOS, ret, r_media, v_media, dd,
    regimes, regime_ids, cluster_ids,
    discretizar_estado_financeiro,
    calcular_recompensa_portfolio,
    aplicar_acao_portfolio
)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
import numpy as np



class PolicyMLP(nn.Module):
    def __init__(self, state_dim, num_actions):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, num_actions)
        )

    def forward(self, x):
        logits = self.net(x)
        return torch.softmax(logits, dim=1)




Arquivo encontrado! Carregando dados locais...
            BOVA11.SA  SMAL11.SA  IVVB11.SA  PETR4.SA   VALE3.SA  ITUB4.SA  \
Date                                                                         
2015-01-01        NaN        NaN        NaN       NaN        NaN       NaN   
2015-01-02  47.259998  52.020000  55.799999  2.572509  10.505502  9.372999   
2015-01-03        NaN        NaN        NaN       NaN        NaN       NaN   
2015-01-04        NaN        NaN        NaN       NaN        NaN       NaN   
2015-01-05  46.320000  50.549999  55.750000  2.352637  10.347521  9.420101   

                   SPY        QQQ         IWM        EEM         GLD  \
Date                                                                   
2015-01-01         NaN        NaN         NaN        NaN         NaN   
2015-01-02  171.093704  94.906532  103.315140  30.789457  114.080002   
2015-01-03         NaN        NaN         NaN        NaN         NaN   
2015-01-04         NaN        NaN         NaN 

d:\python\portfolioDeAtivos\teste.py:154: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  retornos = precos.pct_change().fillna(0)


-0.008030805295683797
Soma: 1.0000000000000002
Min: 0.05767012687427913
Max: 0.07727797001153404
[0.07727797 0.05767013 0.05767013 0.05767013 0.05767013]
Ação 0 | soma=1.000000 | min=0.0577 | max=0.0773
Ação 1 | soma=1.000000 | min=0.0565 | max=0.0761
Ação 2 | soma=1.000000 | min=0.0554 | max=0.0750
Ação 3 | soma=1.000000 | min=0.0543 | max=0.0740
Ação 4 | soma=1.000000 | min=0.0533 | max=0.0729
Ação 5 | soma=1.000000 | min=0.0522 | max=0.0718
Ação 6 | soma=1.000000 | min=0.0512 | max=0.0708
Ação 7 | soma=1.000000 | min=0.0502 | max=0.0698
Ação 8 | soma=1.000000 | min=0.0492 | max=0.0688
Ação 9 | soma=1.000000 | min=0.0483 | max=0.0679


In [2]:
AÇOES_POR_ATIVO = 2
state_dim = NUM_ATIVOS + 3 + NUM_ATIVOS
num_actions = NUM_ATIVOS * AÇOES_POR_ATIVO

taxa_aprendizado = 1e-4
policy_net_final = PolicyMLP(state_dim, num_actions)
optimizer_final = optim.Adam(policy_net_final.parameters(), lr=taxa_aprendizado)


# teste shapes
pesos = np.ones(NUM_ATIVOS)/NUM_ATIVOS
t = 1
state = discretizar_estado_financeiro(
    pesos, regime_ids[t], cluster_ids, r_media.iloc[t - 1], v_media.iloc[t - 1],dd.iloc[t - 1]
)
state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
_ = policy_net_final(state)
print("state shape:", state.shape)
probs = policy_net_final(state)
print("action_probs shape:", probs.shape)
print("sum probs:", probs.sum().item())

state shape: torch.Size([1, 1, 37])
action_probs shape: torch.Size([1, 1, 34])
sum probs: 34.0


C:\Users\olazu\AppData\Local\Temp\ipykernel_22384\3151837496.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)


In [3]:
episodios = len(ret) - 1
pesos = np.ones(NUM_ATIVOS) / NUM_ATIVOS

for ep in range(1,episodios):
    t = ep
    t_obs = t - 1

    regime_id = regime_ids[t_obs]
    
    dd_obs = dd.iloc[t_obs]  # oque robo ver
    r_dia_media = r_media.iloc[t_obs]
    v_dia_media = v_media.iloc[t_obs]

    # Retorno do dia 
    drawdown_dia = dd.iloc[t]
    retornos_dia = ret.iloc[t].values
    
    state = discretizar_estado_financeiro(pesos, regime_id, cluster_ids, r_dia_media, v_dia_media, dd_obs)


    state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)

    action_probs = policy_net_final(state)
    m = Categorical(action_probs)
    action = m.sample().item()
    log_prob = m.log_prob(torch.tensor(action))

    novos_pesos = aplicar_acao_portfolio(pesos, action)

    recompensa = calcular_recompensa_portfolio(
        pesos_antigos=pesos,
        pesos_novos=novos_pesos,
        retornos_dia=retornos_dia,
        drawdown_dia=drawdown_dia,
        lambda_dd=0.2,
        lambda_tc=0.001
    )

    loss = -log_prob * recompensa
    optimizer_final.zero_grad()
    loss.backward()
    optimizer_final.step()

    pesos = novos_pesos

    if ep % 200 == 0:
        print(f"Ep {ep} | reward {recompensa:.5f} | regime {regimes[t]}")


C:\Users\olazu\AppData\Local\Temp\ipykernel_22384\4286855563.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)


Ep 200 | reward -0.00682 | regime neutro
Ep 400 | reward -0.01431 | regime alta_vol
Ep 600 | reward 0.00684 | regime bull
Ep 800 | reward 0.00324 | regime bull
Ep 1000 | reward -0.00073 | regime bull
Ep 1200 | reward -0.00322 | regime alta_vol
Ep 1400 | reward 0.00221 | regime alta_vol
Ep 1600 | reward 0.00119 | regime alta_vol
Ep 1800 | reward 0.00396 | regime neutro
Ep 2000 | reward -0.00010 | regime alta_vol
Ep 2200 | reward 0.02207 | regime bull
Ep 2400 | reward 0.01214 | regime alta_vol
Ep 2600 | reward -0.00706 | regime alta_vol
Ep 2800 | reward -0.00769 | regime bear
Ep 3000 | reward -0.02064 | regime bear
Ep 3200 | reward -0.00054 | regime bear
Ep 3400 | reward -0.00224 | regime alta_vol
Ep 3600 | reward 0.01544 | regime bull
Ep 3800 | reward -0.00354 | regime alta_vol
